# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
cities_df = pd.read_csv("../Weatherpy/cities.csv")
cities_df

In [ ]:
cities_df = cities_df.drop(columns='Unnamed: 0')
cities_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]
clouds = cities_df['Cloudiness']
humidity = cities_df['Humidity']
max_temp = cities_df['Max Temp']
wind_speed = cities_df['Wind Speed']

In [ ]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_weather = cities_df.loc[max_temp < 30].loc[max_temp > 10].loc[wind_speed > 8].loc[clouds > 20]
ideal_weather['Lat'] = pd.to_numeric(ideal_weather['Lat'])
ideal_weather['Lng'] = pd.to_numeric(ideal_weather['Lng'])
ideal_weather

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ""

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type":"lodging","key":g_key,"radius":"5000"}

for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat}, {lng}'

    print(f"Retrieving Results for: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    # print(response)

    try:
        print(f"Closest hotel is {results[0]['name']}.")  
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
hotel_df


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display Map
fig